In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time

# Base URL for company job listings
base_url = 'https://search.jobs.barclays/search-jobs?glat=34.0522&glon=-118.244'

# TODO: Define the pull_company_name function
def pull_company_name(base_url, all_links, job_links):
    """
    Extracts job links from the company careers page.

    Parameters:
    base_url (str): The base URL of the company careers page.
    all_links (list): List of all links on the current page.
    job_links (list): List of collected job links.

    Returns:
    list: Updated list of job links.
    """
    acceptable_locations = [
        'australia',
        'singapore',
        'hong kong',
        'china',
        'canada',
        'united kingdom',
        'united states'
    ]
    url_start = base_url[:base_url.find('search-jobs')]
    result_section = soup.find(id='search-results-list')
    if result_section:
        all_links = result_section.find_all('a', href=True)
        for link in all_links:
            # TODO: Check if the link contains a certain pattern and is not already in job_links
            # Append the link to job_links if it meets the criteria
            href = link.get('href') # extract href
            # check for indicators: 'job', '13015 in hrefs to find proper job_links
            if href and 'job' in href and '13015' in href:
                job_location = link.find_next('div', class_='job-location').text.strip().lower()
                job_url = url_start + href[href.find('job'):]
                # check provided job locations against acceptable locations
                if any(location in job_location for location in acceptable_locations):
                    # check for duplicates before appending to return value
                    if job_url not in job_links:
                        job_links.append(job_url)

    return job_links

def apply_filters(driver):
    job_types = [
        'Graduate',
        'Intern'
    ]
    jobtype_dropdown = driver.find_element(By.ID, 'job_type-toggle')
    jobtype_dropdown.click()
    time.sleep(2)
    first_checkbox = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//input[@data-display="Graduate"]'))
    )
    first_checkbox.click()
    time.sleep(2)
    try:
        jobtype_dropdown.click()
        time.sleep(2)
    except:
        jobtype_dropdown = driver.find_element(By.ID, 'job_type-toggle')
        jobtype_dropdown.click()
        time.sleep(2)

    for job_type in job_types:
        type_checkbox = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, f'//input[@data-display="{job_type}"]'))
        )
        type_checkbox.click()
        time.sleep(1)

    def open_category_dropdown():
        category_button = driver.find_element(By.ID, 'category-toggle')
        if 'expandable-child-open' not in category_button.get_attribute('class'):
            category_button.click()
            time.sleep(2)
    # close category dropdown - page has a quick "reset" upon clicking into category dropdown. must redefine button
    def close_category_dropdown():
        category_button = driver.find_element(By.ID, 'category-toggle')
        if 'expandable-child-open' in category_button.get_attribute('class'):
            category_button.click()
            time.sleep(2)
    # grab checkboxes
    def get_category_checkboxes():
        category_section = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'category-filters-section')))
        return category_section.find_elements(By.CLASS_NAME, 'filter-checkbox')
    
    open_category_dropdown()
    unacceptable_categories = ['hr', 'legal', 'secretarial/admin', 'customer service']
    category_checkboxes = get_category_checkboxes()

    # toggling checkbox "refreshes" page kind of. consequences:
    # 1. deselecting of first checkbox upon clicking into dropdown to select second checkbox
    # 2. cannot iterate through - go with range and index instead
    for i in range(len(category_checkboxes)):
        # scan and toggle each checkbox twice upon checking if checkbox already checked
        for _ in range(2):
                category_checkboxes = get_category_checkboxes()
                category_checkbox = category_checkboxes[i]
                # data-display attribute defines category of dropdown options
                category_data_display = category_checkbox.get_attribute('data-display').lower()
                if category_data_display and category_data_display not in unacceptable_categories and not category_checkbox.is_selected():
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(category_checkbox))
                    category_checkbox.click()
                    time.sleep(2)
                    open_category_dropdown()
    # not necessary but left here incase in block necessary web elements
    close_category_dropdown()

        # def open_country_dropdown():
    #     country_button = driver.find_element(By.ID, 'country-toggle')
    #     if 'expandable-child-open' not in country_button.get_attribute('class'):
    #         country_button.click()
    #         time.sleep(2)
    
    # def close_country_dropdown():
    #     country_button = driver.find_element(By.ID, 'country-toggle')
    #     if 'expandable-child-open' in country_button.get_attribute('class'):
    #         country_button.click()
    #         time.sleep(2)
    # def get_country_checkboxes():
    #     country_section = WebDriverWait(driver, 10). until(
    #         EC.presence_of_element_located((By.ID, 'country-filters-section')))
    #     return country_section.find_elements(By.CLASS_NAME, 'filter-checkbox')
    
    # open_country_dropdown()
    # acceptable_countries = ['canada', 'hong kong china', 'singapore', 'united kingdom', 'united states']
    # country_checkboxes = get_country_checkboxes()

    # for i in range(len(country_checkboxes)):
    #     for _ in range(2):
    #         country_checkboxes = get_country_checkboxes()
    #         country_checkbox = country_checkboxes[i]
    #         country_data_display = country_checkbox.get_attribute('data-display').lower()

    #         if country_data_display and country_data_display in acceptable_countries and not country_checkbox.is_selected():
    #             WebDriverWait(driver, 10).until(EC.element_to_be_clickable(country_checkbox))
    #             country_checkbox.click()
    #             time.sleep(2)
    #             open_country_dropdown()
    # close_country_dropdown()
    # print("Applied filters")
# def apply_filters(driver):

# # NOTE: LIMITED NUMBER OF FILTERS - CANNOT APPLY BOTH COUNTRY AND CATEGORY AT SAME TIME
# #   Filters here do work individually, but with a max number on filter options, applied location filters onto the pull_company_name function
#     # Selenium country filter:
#     # def open_country_dropdown():
#     #     country_button = driver.find_element(By.ID, 'country-toggle')
#     #     if 'expandable-child-open' not in country_button.get_attribute('class'):
#     #         country_button.click()
#     #         time.sleep(2)
    
#     # def close_country_dropdown():
#     #     country_button = driver.find_element(By.ID, 'country-toggle')
#     #     if 'expandable-child-open' in country_button.get_attribute('class'):
#     #         country_button.click()
#     #         time.sleep(2)
#     # def get_country_checkboxes():
#     #     country_section = WebDriverWait(driver, 10). until(
#     #         EC.presence_of_element_located((By.ID, 'country-filters-section')))
#     #     return country_section.find_elements(By.CLASS_NAME, 'filter-checkbox')
    
#     # open_country_dropdown()
#     # acceptable_countries = ['canada', 'hong kong china', 'singapore', 'united kingdom', 'united states']
#     # country_checkboxes = get_country_checkboxes()

#     # for i in range(len(country_checkboxes)):
#     #     for _ in range(2):
#     #         country_checkboxes = get_country_checkboxes()
#     #         country_checkbox = country_checkboxes[i]
#     #         country_data_display = country_checkbox.get_attribute('data-display').lower()

#     #         if country_data_display and country_data_display in acceptable_countries and not country_checkbox.is_selected():
#     #             WebDriverWait(driver, 10).until(EC.element_to_be_clickable(country_checkbox))
#     #             country_checkbox.click()
#     #             time.sleep(2)
#     #             open_country_dropdown()

#     # close_country_dropdown()
#     # open category dropdown

#     def open_category_dropdown():
#         category_button = driver.find_element(By.ID, 'category-toggle')
#         if 'expandable-child-open' not in category_button.get_attribute('class'):
#             category_button.click()
#             time.sleep(2)
#     # close category dropdown - page has a quick "reset" upon clicking into category dropdown. must redefine button
#     def close_category_dropdown():
#         category_button = driver.find_element(By.ID, 'category-toggle')
#         if 'expandable-child-open' in category_button.get_attribute('class'):
#             category_button.click()
#             time.sleep(2)
#     # grab checkboxes
#     def get_category_checkboxes():
#         category_section = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.ID, 'category-filters-section')))
#         return category_section.find_elements(By.CLASS_NAME, 'filter-checkbox')
    
#     open_category_dropdown()
#     unacceptable_categories = ['hr', 'legal', 'secretarial/admin', 'customer service']
#     category_checkboxes = get_category_checkboxes()

#     # toggling checkbox "refreshes" page kind of. consequences:
#     # 1. deselecting of first checkbox upon clicking into dropdown to select second checkbox
#     # 2. cannot iterate through - go with range and index instead
#     for i in range(len(category_checkboxes)):
#         # scan and toggle each checkbox twice upon checking if checkbox already checked
#         for _ in range(2):
#                 category_checkboxes = get_category_checkboxes()
#                 category_checkbox = category_checkboxes[i]
#                 # data-display attribute defines category of dropdown options
#                 category_data_display = category_checkbox.get_attribute('data-display').lower()
#                 if category_data_display and category_data_display not in unacceptable_categories and not category_checkbox.is_selected():
#                     WebDriverWait(driver, 10).until(EC.element_to_be_clickable(category_checkbox))
#                     category_checkbox.click()
#                     time.sleep(2)
#                     open_category_dropdown()
#     # not necessary but left here incase in block necessary web elements
#     close_category_dropdown()
#     # print("Applied filters")

# TODO: Define the button_company_name function
def button_company_name(driver, page, outer_loop_break, all_links):
    """
    Handles pagination by clicking the "next page"/"load more" button.

    Parameters:
    driver (WebDriver): The Selenium WebDriver instance.
    page (int): The current page number. (optional)
    outer_loop_break (bool): Flag to indicate when to stop scraping.
    all_links (list): List of all links on the current page.

    Returns:
    tuple: Updated page number and outer_loop_break flag.
    """

    try:
        next_button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'next'))
        )
        # next button receives "disabled" in class when on last page
        if 'disabled' in next_button.get_attribute('class'):
            outer_loop_break = True
        else:
            next_button.click()
            time.sleep(2)
    except Exception as e:
        print(f"Error occurred while trying to click the 'next page' button: {e}")
        outer_loop_break = True
    return page + 1, outer_loop_break

# WebDriver setup
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get(base_url)


# Initialize variables
job_links = []
prev_page = 0
page = 1

try:
    cookie_accept_all = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.ID, 'system-ialert-button'))
    )
    cookie_accept_all.click()
except Exception as e:
    pass

apply_filters(driver)

# Main scraping loop
while True:
    time.sleep(5)
    outer_loop_break = False
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    all_links = soup.find_all("a", href=True)

    job_links = pull_company_name(base_url, all_links, job_links)
    # Handle pagination
    page, outer_loop_break = button_company_name(driver, page, outer_loop_break, all_links)
    
    if outer_loop_break:
        break

# Output the collected job links
print("Length of job links:", len(job_links), job_links)

# Close the WebDriver
driver.quit()

Length of job links: 64 ['https://search.jobs.barclays/job/greater-manchester/uk-corporate-banking-graduate-programme-2025-manchester/13015/69503597392', 'https://search.jobs.barclays/job/london/international-corporate-banking-graduate-programme-2025-london/13015/69503596176', 'https://search.jobs.barclays/job/london/sales-trading-and-structuring-graduate-programme-2025-london/13015/69503596112', 'https://search.jobs.barclays/job/london/credit-research-graduate-programme-2025-london/13015/69503595616', 'https://search.jobs.barclays/job/london/banking-graduate-programme-2025-london/13015/69503594992', 'https://search.jobs.barclays/job/singapore/sales-trading-and-structuring-graduate-programme-2025-singapore/13015/68650256720', 'https://search.jobs.barclays/job/london/international-corporate-banking-graduate-programme-2025-london/13015/69473757184', 'https://search.jobs.barclays/job/london/uk-corporate-banking-graduate-programme-2025-london/13015/69503596656', 'https://search.jobs.barcla